<a href="https://colab.research.google.com/github/leylaaaa1/MetCompCompl-202320_Canon_Guatibonza/blob/main/Rf_TO_PERUSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetch seismic data from web-service and calculate RF automatically

After Seispy v1.3.0, user can calculate RFs with specified network and station which can be fetched from [FDSN web service](https://www.fdsn.org/webservices/). This section shows a example to calculate PRFs with fetching station and event information from web service.

```{note}
This notebook can be downloaded as **{nb-download}`rf-from-ws.ipynb`**
```

In [ ]:
pip install python-seispy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 941.1/941.1 kB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for pyside6>=6.2.0 from https://files.pythonhosted.org/packages/24/3a/a970808004b16dabdfaf77fa602b43a85c4d8812709a8bae065577283c4c/PySide6-6.6.1-cp38-abi3-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for pyproj from https://files.pythonhosted.org/packages/64/90/dfe5c00de1ca4dbb82606e79790659d4ed7f0ed8d372bccb3baca2a5abe0/pyproj-3.6.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for shiboken6==6.6.1 from https://files.pythonhosted.org/packages/bb/72/e54f758e49e8da0dcd9490d006c41a814b0e56898ce4ca054d60cdba97bd/shiboken6-6.6.1-cp38-abi3-manylinux_2_28_x86_64.whl.metadata
  Obtaining dependency information for PySide6-Essentials==6.6.1 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from seispy.rf import RF
from seispy.io import Query
from obspy import UTCDateTime
#import pytest
import glob

## Get information of stations
Before running this script, we can visually search stations from portal of web-service, such as [GFZ webdc3](http://eida.gfz-potsdam.de/webdc3/) or [IRIS GMap](https://ds.iris.edu/gmap/). The URL of FDSN web service or shortcut names can be found in [obspy.client.fdsn](https://docs.obspy.org/packages/obspy.clients.fdsn.html). The network name, station name, positions, date range, etc. can be found at these services. Now let's fetch station information using these conditions.  
   
The following example illustrates how to request the station information from the Global Seismograph Network("`IU`").


In [ ]:
query = Query(server='IRIS') ## Server is the URL of FDSN web service or a shortcut name in the obspy.client.fdsn.
query.get_stations(network='TO', station='PE*', level='channel')
#print(query.stations)

## Fetch data and calculate RF with different gauss factors
    
   The following example illustrates how to request the `'BH?'` channels, `'00'` location of station Ulaanbaatar (`'ULN'`) of the Global Seismograph Network(`'IU'`) for events between "2013-08-01"  and "2013-10-31" (UTC), calculate the RF with 4 gauss factors simultaneously, and save the raw seismic data and RFs.   
      

<h3 id="rfpara">Set the parameters for matching catalog and estimating RF</h3>  
   
   All parameters for matching catalog and estimating RF are in the [`RF.para`](#rfpara).  These parameters can be set according to user needs.
   Online catalog (`'cata_server'`) is fetched from the FDSN web service client for ObsPy ([obspy.client.fdsn](https://docs.obspy.org/packages/obspy.clients.fdsn.html)).

In [ ]:
rf = RF()
rf.para.data_server = 'IRIS'
rf.para.cata_server = 'IRIS'
rf.para.stainfo.network = 'TO'
rf.para.stainfo.station = 'PF29'
rf.para.stainfo.channel = 'HH?'
rf.para.stainfo.location = '01'
rf.para.datapath = './Data/{}.{}'.format(rf.para.stainfo.network, rf.para.stainfo.station)
rf.para.use_remote_data = True
rf.para.ref_comp ='HHZ'
rf.para.phase = 'P'
rf.para.noisegate = 1
rf.para.magmin = 5.8
rf.para.dismin = 30
rf.para.dismax = 95
rf.para.gauss = [7.0] ##RF with different Gauss factor will be calculated simultaneously.
#rf.para.gauss = [2.5]
rf.para.rmsgate = 0.4
rf.para.freqmin = 0.05
rf.para.freqmax = 2.0
rf.para.comp = 'RTZ'
rf.para.date_begin = UTCDateTime('20080701')
rf.para.date_end = UTCDateTime('20151231')


### load station information and search events  
  - Fetch the station information from the data server ([`data_server`](#rfpara))  
  - Search the event information from the catalog server ([`cata_server`](#rfpara)).  
  Here, we use the `'IRIS'` client. Available catalogs in the IRIS are listed in [IRIS DMC FDSNMS event Web Server](https://service.iris.edu/fdsnws/event/1/catalogs), such as `'ISC'`, `'NEIC PDE'` and `'GCMT'`.

In [ ]:
rf.load_stainfo()
rf.search_eq(catalog='NEIC PDE')
#print(rf.eq_lst) ##The matched event lists are listed.

2023-12-21 16:24:34,224 [RF] INFO: Load station info of TO.PF29 from IRIS web-service
2023-12-21 16:24:35,705 [RF] INFO: TO.PF29, latitude: -14.331, longitude: -71.194
2023-12-21 16:24:35,706 [RF] INFO: Searching earthquakes from IRIS
2023-12-21 16:24:47,771 [RF] INFO: 198 earthquakes are found


### Match catalog and fetch seismic data
  Match events and fetch seismic data with the parameters such as the data type (`'SAC'`)  and dateformat `'%Y.%j.%H.%M.%S'` set in the [`RF.para`](#rfpara).
  

In [ ]:
rf.match_eq()

2023-12-21 16:24:50,671 [RF] INFO: Fetch seismic data from IRIS
2023-12-21 16:24:50,683 [RF] INFO: Fetch waveforms of (1/198) event 2008.209.21.15.42 from IRIS
2023-12-21 16:24:51,383 [RF] ERROR: Error in fetching waveforms of event 2008.209.21.15.42: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:24:51,390 [RF] INFO: Fetch waveforms of (2/198) event 2008.211.20.56.22 from IRIS
2023-12-21 16:24:52,086 [RF] ERROR: Error in fetching waveforms of event 2008.211.20.56.22: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:24:52,094 [RF] INFO: Fetch waveforms of (3/198) event 2008.224.23.38.38 from IRIS
2023-12-21 16:24:52,741 [RF] ERROR: Error in fetching waveforms of event 2008.224.23.38.38: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:24:52,748 [RF] INFO: Fetch waveforms of (4/198) event 2008.241.12.37.35 from IRIS
2023-12-21 16:24:53,395 [RF] E

2023-12-21 16:25:16,909 [RF] ERROR: Error in fetching waveforms of event 2009.155.17.25.25: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:25:16,917 [RF] INFO: Fetch waveforms of (32/198) event 2009.157.20.33.28 from IRIS
2023-12-21 16:25:17,659 [RF] ERROR: Error in fetching waveforms of event 2009.157.20.33.28: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:25:17,663 [RF] INFO: Fetch waveforms of (33/198) event 2009.167.20.05.56 from IRIS
2023-12-21 16:25:18,385 [RF] ERROR: Error in fetching waveforms of event 2009.167.20.05.56: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:25:18,389 [RF] INFO: Fetch waveforms of (34/198) event 2009.184.11.00.14 from IRIS
2023-12-21 16:25:20,263 [RF] ERROR: Error in fetching waveforms of event 2009.184.11.00.14: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16

2023-12-21 16:25:42,539 [RF] INFO: Fetch waveforms of (63/198) event 2010.035.20.20.21 from IRIS
2023-12-21 16:25:44,603 [RF] INFO: Fetch waveforms of (64/198) event 2010.058.19.54.28 from IRIS
2023-12-21 16:25:47,406 [RF] INFO: Fetch waveforms of (65/198) event 2010.066.07.05.24 from IRIS
2023-12-21 16:25:49,866 [RF] INFO: Fetch waveforms of (66/198) event 2010.094.22.40.43 from IRIS
2023-12-21 16:25:52,171 [RF] INFO: Fetch waveforms of (67/198) event 2010.100.06.29.59 from IRIS
2023-12-21 16:25:54,617 [RF] INFO: Fetch waveforms of (68/198) event 2010.101.22.08.12 from IRIS
2023-12-21 16:25:56,905 [RF] INFO: Fetch waveforms of (69/198) event 2010.125.09.38.23 from IRIS
2023-12-21 16:25:59,177 [RF] INFO: Fetch waveforms of (70/198) event 2010.126.11.35.29 from IRIS
2023-12-21 16:26:01,915 [RF] INFO: Fetch waveforms of (71/198) event 2010.136.05.16.10 from IRIS
2023-12-21 16:26:04,329 [RF] INFO: Fetch waveforms of (72/198) event 2010.139.10.30.10 from IRIS
2023-12-21 16:26:07,592 [RF] I

2023-12-21 16:28:45,166 [RF] INFO: Fetch waveforms of (147/198) event 2012.200.04.25.25 from IRIS
2023-12-21 16:28:46,442 [RF] INFO: Fetch waveforms of (148/198) event 2012.211.12.22.11 from IRIS
2023-12-21 16:28:48,191 [RF] INFO: Fetch waveforms of (149/198) event 2012.229.13.24.44 from IRIS
2023-12-21 16:28:49,839 [RF] INFO: Fetch waveforms of (150/198) event 2012.240.04.37.19 from IRIS
2023-12-21 16:28:51,577 [RF] INFO: Fetch waveforms of (151/198) event 2012.243.13.43.25 from IRIS
2023-12-21 16:28:53,221 [RF] INFO: Fetch waveforms of (152/198) event 2012.269.23.45.24 from IRIS
2023-12-21 16:28:54,957 [RF] INFO: Fetch waveforms of (153/198) event 2012.282.06.26.23 from IRIS
2023-12-21 16:28:56,569 [RF] INFO: Fetch waveforms of (154/198) event 2012.302.03.04.08 from IRIS
2023-12-21 16:28:58,327 [RF] INFO: Fetch waveforms of (155/198) event 2012.302.18.54.20 from IRIS
2023-12-21 16:29:00,011 [RF] INFO: Fetch waveforms of (156/198) event 2012.304.02.49.02 from IRIS
2023-12-21 16:29:01,

2023-12-21 16:29:37,577 [RF] INFO: Fetch waveforms of (191/198) event 2013.306.15.52.46 from IRIS
2023-12-21 16:29:38,285 [RF] ERROR: Error in fetching waveforms of event 2013.306.15.52.46: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:29:38,293 [RF] INFO: Fetch waveforms of (192/198) event 2013.317.23.45.47 from IRIS
2023-12-21 16:29:38,833 [RF] ERROR: Error in fetching waveforms of event 2013.317.23.45.47: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:29:38,841 [RF] INFO: Fetch waveforms of (193/198) event 2013.320.03.34.31 from IRIS
2023-12-21 16:29:39,504 [RF] ERROR: Error in fetching waveforms of event 2013.320.03.34.31: No data available for request.
HTTP Status code: 204
Detailed response of server:
2023-12-21 16:29:39,508 [RF] INFO: Fetch waveforms of (194/198) event 2013.321.09.04.55 from IRIS
2023-12-21 16:29:40,130 [RF] ERROR: Error in fetching waveforms of event 2013.321.09.0

### Calculate RF
  - Remove the linear trend (`detrend`) and apply a bandpass filter (`filter`) to the data. The frequencies for the bandpass filter are set in the [`RF.para`](#rfpara) ([`'para.freqmin'`](#rfpara) and [`'para.freqmin'`](#rfpara));   
  - Mark phase arrivals with the server of Taup and the velocity mode ([`'para.velmod'`](#rfpara)) can be set in the [`RF.para`](#rfpara);   
  - Rotate the seismic data to `'RTZ'` or`'LQT'` and delete the events with the SNR lower than the [`'para.noisegate'`](#rfpara);   
  - Save the raw SAC data download from the web server;   
  Trim the RF bewteen the times of [`para.time_before`](#rfpara) and [`para.time_after`](#rfpara);    
  - Do deconvolution to obtain the RFs with different gauss factors. Deconvolution methods (`para.decon_method`) of Time-domain iterative deconvolution (`'iter'`) and frequency-domian water-level deconvolution (`'water'`) are available.  

In [ ]:
rf.detrend()
rf.filter()
rf.cal_phase()
rf.rotate()
rf.drop_eq_snr()
rf.save_raw_data()
rf.trim()
rf.deconv()

2023-12-21 16:30:53,755 [RF] INFO: Detrend all data
2023-12-21 16:30:53,987 [RF] INFO: Filter all data from 0.05 to 2.0
2023-12-21 16:30:54,230 [RF] INFO: Calculate P arrivals and ray parameters for all data
2023-12-21 16:30:54,549 [RF] INFO: Rotate P phase to NE->RT
2023-12-21 16:30:54,568 [RF] INFO: Reject data record with SNR less than 1
/home/sismologia/anaconda3/envs/seispy/lib/python3.11/site-packages/seispy/geo.py:143: RuntimeWarning: divide by zero encountered in log10
  return 10 * np.log10(spow / npow)
2023-12-21 16:30:54,680 [RF] INFO: 92 events left after SNR calculation
2023-12-21 16:30:55,212 [RF] INFO: Trim waveforms from 10.00 before P to 120.00 after P
2023-12-21 16:30:55,857 [RF] INFO: Iterative Decon 2010.027.17.50.34 (1/92) iterations: 400; final RMS: 0.4504
2023-12-21 16:30:56,471 [RF] INFO: Iterative Decon 2010.035.20.30.14 (2/92) iterations: 400; final RMS: 0.3504
2023-12-21 16:30:57,098 [RF] INFO: Iterative Decon 2010.066.07.11.38 (3/92) iterations: 400; final R

2023-12-21 16:31:38,805 [RF] INFO: Iterative Decon 2012.122.22.48.51 (69/92) iterations: 400; final RMS: 0.4733
2023-12-21 16:31:39,423 [RF] INFO: Iterative Decon 2012.131.02.19.58 (70/92) iterations: 400; final RMS: 0.3023
2023-12-21 16:31:40,059 [RF] INFO: Iterative Decon 2012.139.02.05.19 (71/92) iterations: 400; final RMS: 0.3002
2023-12-21 16:31:40,680 [RF] INFO: Iterative Decon 2012.181.15.40.02 (72/92) iterations: 400; final RMS: 0.4394
2023-12-21 16:31:41,309 [RF] INFO: Iterative Decon 2012.211.12.27.24 (73/92) iterations: 400; final RMS: 0.3417
2023-12-21 16:31:41,939 [RF] INFO: Iterative Decon 2012.229.13.29.40 (74/92) iterations: 400; final RMS: 0.2905
2023-12-21 16:31:42,557 [RF] INFO: Iterative Decon 2012.240.04.41.58 (75/92) iterations: 400; final RMS: 0.1694
2023-12-21 16:31:43,175 [RF] INFO: Iterative Decon 2012.243.13.55.05 (76/92) iterations: 400; final RMS: 0.2124
2023-12-21 16:31:43,798 [RF] INFO: Iterative Decon 2012.269.23.53.11 (77/92) iterations: 400; final RMS:

### Save the RFs   
  Save the RFs calculating with different Gauss factors.

In [ ]:
for ff in rf.para.gauss:
    rf.para.rfpath = './RFresult/F{:.1f}/{}.{}'.format(
        ff, rf.para.stainfo.network, rf.para.stainfo.station)
    rf.saverf(ff)

2023-12-21 16:40:03,630 [RF] INFO: Save RFs with final RMS less than 0.40 and criterion of None
2023-12-21 16:40:03,756 [RF] INFO: 67 PRFs are saved.
